# Cenário II - Proposta

O segundo entregável consiste na transformação de dados disponíveis em <a href="https://drive.google.com/file/d/1IDCjpDZh5St97jw4K_bAewJ8hf-rax9C/view?usp=sharing">arquivo Json</a> para o formato de dataframe, algo comum no dia a dia da empresa. Após transformar esse Json em dataframe é possível perceber que a coluna "item_list" está como dicionário. Seu gestor pediu dois pontos de atenção nessa tarefa:

- Expandir a coluna num mesmo dataframe;
- Normalizar os itens dessa coluna de dicionário e dividí-los em dois dataframes separados, seguindo o modelo relacional.


In [103]:
import pandas as pd
import numpy as np

In [104]:
df_main = pd.read_json("../data/data.json")

In [105]:
df_main.head()

,CreateDate,EmissionDate,Discount,NFeNumber,NFeID,ItemList
0,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,501,1,"[{'ProductName': 'Rice', 'Value': 35.55, 'Quan..."
1,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,502,2,"[{'ProductName': 'Tomate', 'Value': 12.25, 'Qu..."
2,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,503,3,"[{'ProductName': 'Beer', 'Value': 9.0, 'Quanti..."


# Verificação das Colunas

Podemos ver que a coluna `ItemList` é composta por uma lista de dicionários, que inforam os produtos, valores e quantidades que foram adquiridos.

In [106]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CreateDate    3 non-null      object
 1   EmissionDate  3 non-null      object
 2   Discount      3 non-null      int64 
 3   NFeNumber     3 non-null      int64 
 4   NFeID         3 non-null      int64 
 5   ItemList      3 non-null      object
dtypes: int64(3), object(3)
memory usage: 272.0+ bytes


In [107]:
df_main["ItemList"]

0    [{'ProductName': 'Rice', 'Value': 35.55, 'Quan...
1    [{'ProductName': 'Tomate', 'Value': 12.25, 'Qu...
2    [{'ProductName': 'Beer', 'Value': 9.0, 'Quanti...
Name: ItemList, dtype: object

In [108]:
df_main["ItemList"][0]

[{'ProductName': 'Rice', 'Value': 35.55, 'Quantity': 2},
 {'ProductName': 'Flour', 'Value': 11.55, 'Quantity': 5},
 {'ProductName': 'Bean', 'Value': 27.15, 'Quantity': 7}]

Primeiro, podemos apenas transformar a informação da lista em diferentes colunas, sendo que cada entrada é o dicionário para um produto.

In [109]:
df_main_extended = pd.concat([df_main.drop(["ItemList"], axis=1), df_main["ItemList"].apply(pd.Series)], axis=1)

In [110]:
df_main_extended.head()

,CreateDate,EmissionDate,Discount,NFeNumber,NFeID,0,1,2
0,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,501,1,"{'ProductName': 'Rice', 'Value': 35.55, 'Quant...","{'ProductName': 'Flour', 'Value': 11.55, 'Quan...","{'ProductName': 'Bean', 'Value': 27.15, 'Quant..."
1,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,502,2,"{'ProductName': 'Tomate', 'Value': 12.25, 'Qua...","{'ProductName': 'Pasta', 'Value': 7.55, 'Quant...",NaN
2,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,503,3,"{'ProductName': 'Beer', 'Value': 9.0, 'Quantit...","{'ProductName': 'French fries', 'Value': 10.99...","{'ProductName': 'Ice cream', 'Value': 27.15, '..."


Nas colunas:
- "CreateDate",
- "EmissionDate",
- "Discount",
- "NFeNumber",
- "NFeID",     

não temos nenhuma informação sobre os produtos. Podemos fazer uma seleção nestas colunas de modo que elas fiquem de fora das próximas etapas de processamento, em que iremos tirar as colunas dos produtos e passar para DataFrame separado fazendo o link pela coluna `NFeID`.

In [111]:
columns_main = ["CreateDate", "EmissionDate", "Discount", "NFeNumber", "NFeID"]

In [122]:
list_product = [] 
list_product_col = []
for i, row in enumerate(df_main["ItemList"]):
    for item in row:
        item["NFeID"] = df_main["NFeID"][i]
        list_product.append(item)
    list_product_col.append(i)

In [113]:
list_product

[{'ProductName': 'Rice', 'Value': 35.55, 'Quantity': 2, 'NFeID': 1},
 {'ProductName': 'Flour', 'Value': 11.55, 'Quantity': 5, 'NFeID': 1},
 {'ProductName': 'Bean', 'Value': 27.15, 'Quantity': 7, 'NFeID': 1},
 {'ProductName': 'Tomate', 'Value': 12.25, 'Quantity': 10, 'NFeID': 2},
 {'ProductName': 'Pasta', 'Value': 7.55, 'Quantity': 5, 'NFeID': 2},
 {'ProductName': 'Beer', 'Value': 9.0, 'Quantity': 6, 'NFeID': 3},
 {'ProductName': 'French fries', 'Value': 10.99, 'Quantity': 2, 'NFeID': 3},
 {'ProductName': 'Ice cream', 'Value': 27.15, 'Quantity': 1, 'NFeID': 3}]

In [114]:
df_prod = pd.DataFrame(list_product)

In [115]:
df_prod

,ProductName,Value,Quantity,NFeID
0,Rice,35.55,2,1
1,Flour,11.55,5,1
2,Bean,27.15,7,1
3,Tomate,12.25,10,2
4,Pasta,7.55,5,2
5,Beer,9.00,6,3
6,French fries,10.99,2,3
7,Ice cream,27.15,1,3


In [119]:
list_product_col

['0', '1', '2']

In [126]:
df_main.drop("ItemList", axis=1, inplace=True)

In [127]:
df_main

,CreateDate,EmissionDate,Discount,NFeNumber,NFeID
0,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,501,1
1,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,502,2
2,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,503,3


In [128]:
df_prod

,ProductName,Value,Quantity,NFeID
0,Rice,35.55,2,1
1,Flour,11.55,5,1
2,Bean,27.15,7,1
3,Tomate,12.25,10,2
4,Pasta,7.55,5,2
5,Beer,9.00,6,3
6,French fries,10.99,2,3
7,Ice cream,27.15,1,3


Por último, temos dois DataFrames:
- df_main,
- df_prod,

nas quais temos o dado de entrada em df_main com exceção da coluna `ItemList`, e o dado dos produtos para cada nota em df_prod. 